In [13]:
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:

import pandas as pd
import numpy as np
import faiss  # IMPORTANTE: Biblioteca FAISS para otimização
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from tqdm import tqdm
from collections import Counter

In [ ]:
# ==========================================
# 1. Carregamento e Limpeza 
# ==========================================
csv_files = [
    "novods/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    #caso queira carregar os outros arquivos, só descomentar,
    #mas vai precisar alterar os testes para refletir a realidade dos ataques majoritarios e minoritarios
    #"novods/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv", s
    #"novods/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    #"novods/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    #"novods/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv",
]

dataframes_list = []
for file_path in csv_files: #ler os arquivos
    df_temp = pd.read_csv(file_path, low_memory=False) #salvar em um dataframe temporario
    df_temp.drop_duplicates(inplace=True) #dropar as duplicatas
    dataframes_list.append(df_temp) #lista dos dataframes

print("Concatenando datasets...") #concatenar os datasets
dataset_ids = pd.concat(dataframes_list, ignore_index=True) #
dataset_ids.columns = dataset_ids.columns.str.strip() 

# Seleção de Features
feature_columns = [
    "Label", "Protocol", "Fwd Header Len", "Bwd Header Len", "Flow IAT Mean",
    "Idle Mean", "Bwd Pkt Len Mean", "Fwd Pkt Len Mean", "Bwd IAT Mean", "Fwd IAT Mean",
    "Active Mean", "SYN Flag Cnt", "PSH Flag Cnt", "ACK Flag Cnt", "URG Flag Cnt",
    "FIN Flag Cnt", "Fwd Pkt Len Min", "Flow IAT Min", "Pkt Len Min", "Fwd Pkt Len Max",
    "Flow IAT Max", "Pkt Len Max", "Active Max", "Active Min"
]

features = dataset_ids[feature_columns].copy()

# Criação do Flow_Label (ID único do fluxo)
columns_to_concat = [col for col in feature_columns if col not in ['Label', 'Protocol']]
features["Flow_Label"] = features[columns_to_concat].astype(str).agg('-'.join, axis=1)
features["Flow_Label"] = features["Flow_Label"].str.lower().str.strip()

# Preparação Final (Remover duplicatas de Flow_Label para garantir unicidade do "usuário")
features = features.drop_duplicates(subset=['Flow_Label'])
features = features.set_index("Flow_Label")

# Separar X (features numéricas) e y (Rótulo)
X_raw = features.drop(columns=['Label', 'Protocol'], errors='ignore')
y_raw = features['Label']

print("Sanitizando dados numéricos (removendo cabeçalhos repetidos e strings inválidas)...")
# Força a conversão para numérico. Se encontrar string ('Fwd Header Len'), vira NaN.
X_raw = X_raw.apply(pd.to_numeric, errors='coerce')

# Verifica quais linhas têm NaN (eram as linhas com lixo/cabeçalho)
rows_with_nan = X_raw.isnull().any(axis=1)
if rows_with_nan.sum() > 0:
    print(f"   AVISO: Removendo {rows_with_nan.sum()} linhas inválidas que não puderam ser convertidas para float.")
    # Remove essas linhas tanto do X quanto do y para manter a consistência
    X_raw = X_raw[~rows_with_nan]
    y_raw = y_raw[~rows_with_nan]

print("Dados sanitizados com sucesso.")

# --- NOVO: Descrição do Dataset (Pós-Sanitização) ---
print("\n" + "="*40)
print("       RESUMO DO DATASET (Limpo)       ")
print("="*40)
print(f"Shape Total: {X_raw.shape} (Linhas, Colunas)")
print("\nDistribuição de Rótulos (Top 20):")
print(y_raw.value_counts().head(20))
print("\nInformações das Colunas:")
print(X_raw.info(verbose=False))
print("="*40 + "\n")

Concatenando datasets...
Sanitizando dados numéricos (removendo cabeçalhos repetidos e strings inválidas)...
   AVISO: Removendo 1 linhas inválidas que não puderam ser convertidas para float.
Dados sanitizados com sucesso.

       RESUMO DO DATASET (Limpo)       
Shape Total: (2989048, 22) (Linhas, Colunas)

Distribuição de Rótulos (Top 20):
Label
Benign                      2596445
DoS attacks-Hulk             145009
Bot                          143967
Infilteration                 51834
DoS attacks-GoldenEye         41356
DoS attacks-Slowloris          9859
Brute Force -Web                337
Brute Force -XSS                143
DoS attacks-SlowHTTPTest         55
SQL Injection                    43
Name: count, dtype: int64

Informações das Colunas:
<class 'pandas.core.frame.DataFrame'>
Index: 2989048 entries, 192-152-9425.666667-0.0-539.0-61.44444444-8569.5-17673.125-0.0-0-1-0-0-0-0-1.0-0-202-73403.0-1460-0.0-0.0 to 372-352-3436975.32352941-58082282.0-309.7058823529-59.2222222222-72

In [16]:

# ==========================================
# 2. Classe Otimizada com FAISS
# ==========================================
class FAISS_IDS_Recommender:
    def __init__(self, k_neighbors=50):
        self.k = k_neighbors
        self.index = None
        self.train_labels = None
        self.scaler = StandardScaler()
    
    def fit(self, X_train, y_train):
        print(f"   [FAISS] Iniciando fit com {len(X_train)} amostras...")
        if len(X_train) == 0:
            raise ValueError("O conjunto de treino está vazio! Verifique a divisão dos dados.")

        # 1. Scaling
        X_scaled = self.scaler.fit_transform(X_train).astype('float32')
        
        # --- CORREÇÃO: Garantir array C-Contiguous para o FAISS ---
        X_scaled = np.ascontiguousarray(X_scaled)
        # ---------------------------------------------------------
        
        # 2. Normalização L2
        faiss.normalize_L2(X_scaled)
        
        # 3. Criação do Índice FAISS
        d = X_scaled.shape[1]
        self.index = faiss.IndexFlatIP(d)
        self.index.add(X_scaled)
        
        # 4. Lookup Array
        self.train_labels = y_train.to_numpy()
        print("   [FAISS] Índice construído.")
        
    def predict(self, X_test):
        print(f"   [FAISS] Iniciando busca para {len(X_test)} amostras de teste...")
        if len(X_test) == 0:
            return []

        # 1. Preprocessamento do Teste
        X_test_scaled = self.scaler.transform(X_test).astype('float32')

        # --- CORREÇÃO: Garantir array C-Contiguous para o FAISS ---
        X_test_scaled = np.ascontiguousarray(X_test_scaled)
        # ---------------------------------------------------------

        faiss.normalize_L2(X_test_scaled)
        
        # 2. Busca
        # Se k for maior que o número de amostras no treino, reduz k
        k_search = min(self.k, len(self.train_labels))
        D, I = self.index.search(X_test_scaled, k_search)
        
        # 3. Atribuição de Rótulos
        neighbors_labels = self.train_labels[I]
        
        # 4. Votação
        classified_attacks = []
        for row_labels in tqdm(neighbors_labels, desc="   [FAISS] Classificando"):
            counts = Counter(row_labels)
            # Retorna lista ordenada por score
            sorted_attacks = [lbl for lbl, cnt in counts.most_common()]
            classified_attacks.append(sorted_attacks)
            
        return classified_attacks


In [17]:

# ==========================================
# 3. Função de Execução dos Cenários
# ==========================================
def run_test_scenario(test_name, train_split_map, full_data_X, full_data_y):
    print(f"\n{'='*60}\nIniciando {test_name}\n{'='*60}")
    
    train_idxs = []
    test_idxs = []
    
    # --- Divisão de Dados (Customizada por Classe) ---
    unique_labels = full_data_y.unique()
    
    # Verifica quais labels existem no split map, se não tiver, assume 0.3 (default)
    # Se o split_map for vazio, trata de forma especial (Teste 3)
    
    for label in unique_labels:
        # Pega índices dessa classe
        indices = full_data_y[full_data_y == label].index
        
        if len(indices) < 2:
            # Se só tem 1 amostra, vai pro treino (segurança)
            train_idxs.extend(indices)
            continue
            
        test_size = train_split_map.get(label, 0.3) # Default 30% teste
        
        # Se test_size for > 1, assume-se que é contagem absoluta? Não, vamos assumir ratio.
        # Caso especial para Teste 3 onde o split é global ou diferente
        
        try:
            train_i, test_i = train_test_split(indices, test_size=test_size, random_state=42)
            train_idxs.extend(train_i)
            test_idxs.extend(test_i)
        except ValueError:
            # Fallback para classes muito pequenas
            train_idxs.extend(indices)

    # Criação dos DataFrames Finais
    X_train = full_data_X.loc[train_idxs]
    y_train = full_data_y.loc[train_idxs]
    
    # Ajuste para o Teste 3 (onde Teste = Todo o dataset, ou 90%)
    if "Teste 3" in test_name:
        # No Teste 3 do artigo: Treino 10%, Teste 90%
        # A lógica acima já dividiu. Se o map passou 0.9 para teste, está correto.
        X_test = full_data_X.loc[test_idxs]
        y_test = full_data_y.loc[test_idxs]
    elif "Teste 2" in test_name:
         # No Teste 2: Treino só tem algumas classes, Teste tem todas
         # A lógica acima monta teste baseada no complemento do treino.
         # Para garantir que o teste tenha TUDO que não é treino:
         X_test = full_data_X.loc[test_idxs]
         y_test = full_data_y.loc[test_idxs]
    else:
        X_test = full_data_X.loc[test_idxs]
        y_test = full_data_y.loc[test_idxs]

    print(f"Tamanho Treino: {len(X_train)} | Tamanho Teste: {len(X_test)}")
    
    # --- Execução do Modelo FAISS ---
    recommender = FAISS_IDS_Recommender(k_neighbors=50)
    recommender.fit(X_train, y_train)
    y_pred_lists = recommender.predict(X_test)
    
    # --- Avaliação ---
    # Prepara dados para avaliação Multi-Label
    # y_test é uma Series de strings, precisamos converter para lista de listas para o MLB
    y_true_lists = [[label] for label in y_test]
    
    # O y_pred_lists já é uma lista de listas (recomendações)
    # Mas para avaliação estrita (acertou o ataque exato?), pegamos o Top-1
    # O artigo menciona "relatórios de classificação multi-label"
    
    # Coletar todas as classes possíveis
    all_classes = sorted(list(set(y_train.unique()) | set(y_test.unique())))
    mlb = MultiLabelBinarizer(classes=all_classes)
    
    # Para bater com as tabelas do artigo, geralmente se avalia se o Rótulo Real estava nas recomendações
    # Ou se a recomendação Top-1 bate. Vamos assumir Top-1 para precisão direta ou 
    # usar a saída completa do recommender como "predição".
    
    # Ajuste: O artigo compara "Label Real" vs "Classificação".
    # Como é um IDS, vamos considerar a predição principal (índice 0 da lista recomendada)
    y_pred_top1 = [[preds[0]] if preds else [] for preds in y_pred_lists]
    
    y_true_bin = mlb.fit_transform(y_true_lists)
    y_pred_bin = mlb.transform(y_pred_top1) # Avaliando Top-1
    
    print(f"\n--- Relatório de Classificação (Top-1 Recommendation) para {test_name} ---")
    report = classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_, zero_division=0)
    print(report)


In [ ]:

# ==========================================
# 4. Definição e Execução dos Testes
# ==========================================

# CENÁRIO 1: (70% Treino / 30% Teste)
# Todos as classes divididas igual
map_1 = {label: 0.3 for label in y_raw.unique()}
run_test_scenario("Teste 1: 70% Treino / 30% Teste)", map_1, X_raw, y_raw)

# CENÁRIO 2: Ataques Menos Comuns
# selecionados os 5 ataques menos frequentes 70% treino 30% teste"
minority_attacks = [
    'Brute Force -Web', 'Brute Force -XSS', 'DoS attacks-SlowHTTPTest', 
    'DoS attacks-Slowloris', 'SQL Injection'
]

# Filtrando dados
mask_minority = y_raw.isin(minority_attacks)
X_minor = X_raw[mask_minority]
y_minor = y_raw[mask_minority]
map_2 = {label: 0.3 for label in y_minor.unique()}

run_test_scenario("Teste 2: Ataques Menos Comuns", map_2, X_minor, y_minor)


# CENÁRIO 3: Teste 2 do Artigo (Treino Desbalanceado, Teste Total)
# "Treino composto exclusivamente por amostras dos 3 ataques com maior número... 
# divisão de 25% para treino (desses 3) e o resto teste"
major_attacks = ['DoS attacks-Hulk', 'Infilteration', 'Bot'] 
map_3 = {}
for label in y_raw.unique():
    if label in major_attacks:
        map_3[label] = 0.75 # 25% Treino, 75% Teste
    else:
        map_3[label] = 1.0 # 0% Treino (100% Teste) - Simula que não conhece os outros

run_test_scenario("Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset", map_3, X_raw, y_raw)


# CENÁRIO 4: Teste 3 do Artigo (10% Treino, 90% Teste Global)
map_4 = {label: 0.90 for label in y_raw.unique()} # 10% Treino, 90% Teste
run_test_scenario("Teste 4: 10% Treino / 90% Teste (Todos os Dados)", map_4, X_raw, y_raw)


Iniciando Teste 1: 70% Treino / 30% Teste)
Tamanho Treino: 2092329 | Tamanho Teste: 896719
   [FAISS] Iniciando fit com 2092329 amostras...
   [FAISS] Índice construído.
   [FAISS] Iniciando busca para 896719 amostras de teste...


   [FAISS] Classificando: 100%|██████████| 896719/896719 [00:02<00:00, 312870.32it/s]



--- Relatório de Classificação (Top-1 Recommendation) para Teste 1: 70% Treino / 30% Teste) ---
                          precision    recall  f1-score   support

                  Benign       0.98      1.00      0.99    778934
                     Bot       1.00      0.99      1.00     43191
        Brute Force -Web       0.92      0.24      0.38       102
        Brute Force -XSS       1.00      0.40      0.57        43
   DoS attacks-GoldenEye       0.98      1.00      0.99     12407
        DoS attacks-Hulk       1.00      1.00      1.00     43503
DoS attacks-SlowHTTPTest       0.71      1.00      0.83        17
   DoS attacks-Slowloris       0.96      0.99      0.98      2958
           Infilteration       0.84      0.10      0.17     15551
           SQL Injection       0.00      0.00      0.00        13

               micro avg       0.98      0.98      0.98    896719
               macro avg       0.84      0.67      0.69    896719
            weighted avg       0.98      0.

   [FAISS] Classificando: 100%|██████████| 3133/3133 [00:00<00:00, 214690.15it/s]


--- Relatório de Classificação (Top-1 Recommendation) para Teste 2: Ataques Menos Comuns ---
                          precision    recall  f1-score   support

        Brute Force -Web       0.80      0.93      0.86       102
        Brute Force -XSS       0.94      0.67      0.78        43
DoS attacks-SlowHTTPTest       1.00      1.00      1.00        17
   DoS attacks-Slowloris       1.00      1.00      1.00      2958
           SQL Injection       0.00      0.00      0.00        13

               micro avg       0.99      0.99      0.99      3133
               macro avg       0.75      0.72      0.73      3133
            weighted avg       0.99      0.99      0.99      3133
             samples avg       0.99      0.99      0.99      3133


Iniciando Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset


Tamanho Treino: 2733439 | Tamanho Teste: 255609
   [FAISS] Iniciando fit com 2733439 amostras...
   [FAISS] Índice construído.
   [FAISS] Iniciando busca para 255609 amostras de teste...


   [FAISS] Classificando: 100%|██████████| 255609/255609 [00:01<00:00, 242170.82it/s]



--- Relatório de Classificação (Top-1 Recommendation) para Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset ---
                          precision    recall  f1-score   support

                  Benign       0.00      0.00      0.00         0
                     Bot       1.00      0.99      1.00    107976
        Brute Force -Web       0.00      0.00      0.00         0
        Brute Force -XSS       0.00      0.00      0.00         0
   DoS attacks-GoldenEye       0.00      0.00      0.00         0
        DoS attacks-Hulk       1.00      1.00      1.00    108757
DoS attacks-SlowHTTPTest       0.00      0.00      0.00         0
   DoS attacks-Slowloris       0.00      0.00      0.00         0
           Infilteration       1.00      0.07      0.13     38876
           SQL Injection       0.00      0.00      0.00         0

               micro avg       0.85      0.85      0.85    255609
               macro avg       0.30      0.21      0.21    255609
  